In [1]:
# libraries
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [2]:
# read csv
df = pd.read_csv('../input/dhanvarsha/Interview_round_dataset.csv') 

In [3]:
# check dataframe
df.head()

,Sr_No,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,Feature_26,Feature_27,Feature_28,Feature_29,Target_flag
0,26791,-1.101444,0.780847,-0.150606,0.044194,2.779388,3.826857,0.505011,0.448779,0.428350,...,-0.318102,-0.110152,-0.045112,1.008263,0.237554,-0.296286,0.072267,-0.072137,20.13,0
1,91547,0.905274,0.080784,1.547836,2.570243,-0.100630,2.017006,-0.840341,0.709949,-0.016073,...,0.064652,0.504713,0.201644,-0.618860,-0.043392,0.062233,0.117348,0.026562,6.08,0
2,281304,2.021258,0.153611,-1.710915,1.198175,0.602164,-0.652300,0.488583,-0.218795,0.048074,...,0.069765,0.385032,-0.042500,-0.405255,0.455746,-0.484028,-0.021226,-0.077854,1.99,0
3,83873,1.191927,-0.039543,0.537142,0.697681,-0.402698,-0.052089,-0.224193,0.045363,0.317585,...,-0.069138,0.024003,-0.133153,0.087927,0.580386,0.411193,-0.015511,0.000913,12.31,0
4,252334,0.121001,1.015667,-0.404468,-0.539553,0.988905,-0.848640,0.959057,-0.120624,-0.082330,...,-0.337542,-0.819971,0.094267,0.566302,-0.407398,0.112873,0.223010,0.085447,1.79,0


In [4]:
# check shape of dataframe
df.shape

(227198, 31)

In [5]:
# check columns of dataframe
df.columns

Index(['Sr_No', 'Feature_1', 'Feature_2', 'Feature_3', 'Feature_4',
       'Feature_5', 'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9',
       'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13', 'Feature_14',
       'Feature_15', 'Feature_16', 'Feature_17', 'Feature_18', 'Feature_19',
       'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23', 'Feature_24',
       'Feature_25', 'Feature_26', 'Feature_27', 'Feature_28', 'Feature_29',
       'Target_flag'],
      dtype='object')

In [6]:
# check info of dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227198 entries, 0 to 227197
Data columns (total 31 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Sr_No        227198 non-null  int64  
 1   Feature_1    227198 non-null  float64
 2   Feature_2    227198 non-null  float64
 3   Feature_3    227198 non-null  float64
 4   Feature_4    227198 non-null  float64
 5   Feature_5    227198 non-null  float64
 6   Feature_6    227198 non-null  float64
 7   Feature_7    227198 non-null  float64
 8   Feature_8    227198 non-null  float64
 9   Feature_9    227198 non-null  float64
 10  Feature_10   227198 non-null  float64
 11  Feature_11   227198 non-null  float64
 12  Feature_12   227198 non-null  float64
 13  Feature_13   227198 non-null  float64
 14  Feature_14   227198 non-null  float64
 15  Feature_15   227198 non-null  float64
 16  Feature_16   227198 non-null  float64
 17  Feature_17   227198 non-null  float64
 18  Feature_18   227198 non-

In [7]:
# check number of unique values in each column
df.nunique(axis=0)

Sr_No          227198
Feature_1      220792
Feature_2      220792
Feature_3      220792
Feature_4      220794
Feature_5      220794
Feature_6      220790
Feature_7      220791
Feature_8      220784
Feature_9      220793
Feature_10     220785
Feature_11     220787
Feature_12     220791
Feature_13     220795
Feature_14     220792
Feature_15     220791
Feature_16     220785
Feature_17     220785
Feature_18     220792
Feature_19     220790
Feature_20     220778
Feature_21     220768
Feature_22     220787
Feature_23     220770
Feature_24     220789
Feature_25     220785
Feature_26     220786
Feature_27     220756
Feature_28     220721
Feature_29      29283
Target_flag         2
dtype: int64

In [8]:
# check target variable distribution
df['Target_flag'].value_counts()

0    226806
1       392
Name: Target_flag, dtype: int64

As seen from this cell, the dataset is highly imbalanced. While working on an imbalanced dataset accuracy is not an appropriate measure to evaluate model performance. For eg: A classifier which achieves an accuracy of 98 % with an event rate of 2 % is not accurate, if it classifies all instances as the majority class. And eliminates the 2 % minority class observations as noise.

# Baseline Model

We start by creating a base model.

In [9]:
# seperate the features and target variables
labels = df.columns[1:30]
X = df[labels]
y = df['Target_flag']

# create train-test split with stratify  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7, stratify=y)

In [10]:
# check shape of created train-test split
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(204478, 29)
(204478,)
(22720, 29)
(22720,)


In [11]:
# define the different metrics we will be using
metrics = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

# define batch size
batch_size = 2048

In general, you should strive for a small batch size (e.g. 32). Our case is a bit specific, we have highly imbalanced data, so we’ll give a fair chance to each batch before the weights are updated.

In [12]:
# define our model
def build_model(train_data, metrics=metrics):
    
    model = keras.Sequential([
    keras.layers.Dense(units=29,activation='relu',input_shape=(train_data.shape[-1],)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(units=29,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(units=29,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(units=1, activation='sigmoid'),
    ])
    
    # compile our model
    model.compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=metrics
    )

    return model

In [13]:
# training the model
model = build_model(X_train)
history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=20,
    validation_split=0.05,
    shuffle=True,
    verbose=1
)

Epoch 1/20
95/95 [==============================] - 2s 17ms/step - loss: 0.6141 - tp: 176.0000 - fp: 46022.0000 - tn: 147898.0000 - fn: 158.0000 - accuracy: 0.7623 - precision: 0.0038 - recall: 0.5269 - auc: 0.6217 - val_loss: 0.3774 - val_tp: 10.0000 - val_fp: 5.0000 - val_tn: 10200.0000 - val_fn: 9.0000 - val_accuracy: 0.9986 - val_precision: 0.6667 - val_recall: 0.5263 - val_auc: 0.7773
Epoch 2/20
95/95 [==============================] - 1s 10ms/step - loss: 0.3044 - tp: 120.0000 - fp: 5156.0000 - tn: 188764.0000 - fn: 214.0000 - accuracy: 0.9724 - precision: 0.0227 - recall: 0.3593 - auc: 0.5793 - val_loss: 0.1517 - val_tp: 10.0000 - val_fp: 1.0000 - val_tn: 10204.0000 - val_fn: 9.0000 - val_accuracy: 0.9990 - val_precision: 0.9091 - val_recall: 0.5263 - val_auc: 0.7088
Epoch 3/20
95/95 [==============================] - 1s 8ms/step - loss: 0.1281 - tp: 117.0000 - fp: 1328.0000 - tn: 192592.0000 - fn: 217.0000 - accuracy: 0.9920 - precision: 0.0810 - recall: 0.3503 - auc: 0.5967 - 

In [14]:
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 29)                870       
_________________________________________________________________
batch_normalization (BatchNo (None, 29)                116       
_________________________________________________________________
dropout (Dropout)            (None, 29)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 29)                870       
_________________________________________________________________
batch_normalization_1 (Batch (None, 29)                116       
_________________________________________________________________
dropout_1 (Dropout)          (None, 29)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 29)                8

In [15]:
# evaluate model performance on unseen data
model.evaluate(X_test, y_test, batch_size=batch_size)

12/12 [==============================] - 0s 10ms/step - loss: 0.0054 - tp: 23.0000 - fp: 2.0000 - tn: 22679.0000 - fn: 16.0000 - accuracy: 0.9992 - precision: 0.9200 - recall: 0.5897 - auc: 0.9226


[0.00541849248111248,
 23.0,
 2.0,
 22679.0,
 16.0,
 0.9992077350616455,
 0.9200000166893005,
 0.5897436141967773,
 0.9225940704345703]

After trying a lot of combinations of different hyperparameters, the model performance is not that great in terms of different metrics. Let's explore weighted model next.

# Weighted Model

We have more examples of negative class compared to the positive class. Let’s force our model to pay attention to the underrepresented class. We can do that by passing weights for each class.

In [16]:
# get count of positive and negative class
negative_class, positive_class = np.bincount(df.Target_flag)
print("Count of Negative Instance, Positive Instance :", negative_class, positive_class)

# get number of total examples
total_count = len(df.Target_flag)
print("Total Examples :", total_count)

# set weight for majority class
weight_negative = (1 / negative_class) * (total_count) / 2.0
print("Weight for Majority Class :", weight_negative)

# set weight for minority class
weight_positive = (1 / positive_class) * (total_count) / 2.0
print("Weight for Minority Class :", weight_positive)

class_weights = {0: weight_negative, 1: weight_positive}

Count of Negative Instance, Positive Instance : 226806 392
Total Examples : 227198
Weight for Majority Class : 0.5008641746691005
Weight for Minority Class : 289.79336734693874


In [17]:
# training the model
model = build_model(X_train, metrics=metrics)

history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=20,
    validation_split=0.05,
    shuffle=True,
    verbose=1,
    class_weight=class_weights
)

Epoch 1/20
95/95 [==============================] - 2s 18ms/step - loss: 0.6112 - tp: 284.0000 - fp: 49241.0000 - tn: 167360.0000 - fn: 89.0000 - accuracy: 0.7726 - precision: 0.0057 - recall: 0.7614 - auc: 0.8296 - val_loss: 0.4162 - val_tp: 12.0000 - val_fp: 474.0000 - val_tn: 9731.0000 - val_fn: 7.0000 - val_accuracy: 0.9530 - val_precision: 0.0247 - val_recall: 0.6316 - val_auc: 0.8495
Epoch 2/20
95/95 [==============================] - 1s 9ms/step - loss: 0.4466 - tp: 269.0000 - fp: 26374.0000 - tn: 167546.0000 - fn: 65.0000 - accuracy: 0.8639 - precision: 0.0101 - recall: 0.8054 - auc: 0.8934 - val_loss: 0.2599 - val_tp: 13.0000 - val_fp: 276.0000 - val_tn: 9929.0000 - val_fn: 6.0000 - val_accuracy: 0.9724 - val_precision: 0.0450 - val_recall: 0.6842 - val_auc: 0.8764
Epoch 3/20
95/95 [==============================] - 1s 8ms/step - loss: 0.3630 - tp: 280.0000 - fp: 18493.0000 - tn: 175427.0000 - fn: 54.0000 - accuracy: 0.9045 - precision: 0.0149 - recall: 0.8383 - auc: 0.9260 - 

In [18]:
# evaluate model performance on unseen data
model.evaluate(X_test, y_test, batch_size=batch_size)

12/12 [==============================] - 0s 5ms/step - loss: 0.0610 - tp: 33.0000 - fp: 155.0000 - tn: 22526.0000 - fn: 6.0000 - accuracy: 0.9929 - precision: 0.1755 - recall: 0.8462 - auc: 0.9878


[0.06097414717078209,
 33.0,
 155.0,
 22526.0,
 6.0,
 0.9929137229919434,
 0.17553190886974335,
 0.8461538553237915,
 0.9877967834472656]

After trying a lot of combinations of different hyperparameters,although there has been considerable improvement in other metrics, the precision of the model is found to be extremely low.

# Oversample Minority Class

Over-Sampling increases the number of instances in the minority class by randomly replicating them in order to present a higher representation of the minority class in the sample.

In [19]:
# seperate the features and target variables
labels = df.columns[1:30]
X = df[labels]
y = df['Target_flag']

# create train-test split with stratify  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7, stratify=y)
X = pd.concat([X_train, y_train], axis=1)

# seperate the majority and minority classes
negative_class = X[X.Target_flag == 0]
positive_class = X[X.Target_flag == 1]

In [20]:
# length of minority class
print("Length of Minority Class :", len(positive_class))

Length of Minority Class : 353


In [21]:
# upsampling minority class
positive_upsampled = resample(positive_class,
                          replace=True,
                          n_samples=len(negative_class),
                          random_state=7)

In [22]:
# length of upsampled  minority class
print("Length of Upsampled Minority Class :", len(positive_upsampled))

Length of Upsampled Minority Class : 204125


In [23]:
# combine negative class and upsampled positive class
oversampled_data = pd.concat([negative_class, positive_upsampled], axis=0)

In [24]:
# check new data
oversampled_data

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,Feature_26,Feature_27,Feature_28,Feature_29,Target_flag
22186,-0.923124,-0.341242,-1.207207,-4.585854,1.033464,-0.348103,0.375391,0.476410,0.010088,-1.615664,...,0.020497,0.595268,0.014555,-0.349121,-0.284302,-0.950774,0.396593,0.132040,31.20,0
221886,-2.012659,-0.195138,2.002589,-0.536759,-0.299436,-0.102073,-1.214421,-2.373126,0.437249,-1.480027,...,-1.480635,0.423519,0.985501,0.661672,0.037246,0.817614,0.332701,0.000969,30.00,0
56887,0.676771,-1.350408,0.694895,0.221955,-1.175833,0.639744,-0.620011,0.383131,1.168366,-0.395008,...,-0.046101,-0.499162,-0.139720,-0.263602,-0.064527,0.954492,-0.077430,0.036478,251.99,0
62117,-0.639618,0.560891,1.487671,-1.534404,-0.569355,0.239651,-0.416560,0.714844,0.676186,-1.638879,...,0.173382,0.632067,-0.177002,-0.258803,0.065025,-0.695940,0.014810,-0.036132,1.00,0
9214,-1.407331,1.607340,0.464506,-0.334922,0.087982,-0.851665,0.467961,0.574703,-0.767013,-1.337393,...,-0.139035,-0.583866,-0.338291,-0.097976,0.748038,-0.338749,-0.331576,-0.099529,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164077,0.364377,1.443523,-2.220907,2.036985,-1.237055,-1.728161,-2.058582,0.358895,-1.393306,-3.505790,...,0.402730,-0.132129,-0.032977,0.460861,0.560404,0.409366,0.539668,0.296918,0.76,1
200118,-1.548788,1.808698,-0.953509,2.213085,-2.015728,-0.913457,-2.356013,1.197169,-1.678374,-3.538650,...,0.855138,0.774745,0.059037,0.343200,-0.468938,-0.278338,0.625922,0.395573,76.94,1
125611,-2.740483,3.658095,-4.110636,5.340242,-2.666775,-0.092782,-4.388699,-0.280133,-2.821895,-4.466284,...,2.417495,-0.097712,0.382155,-0.154757,-0.403956,0.277895,0.830062,0.218690,112.33,1
72212,-3.975939,-1.244939,-3.707414,4.544772,4.050676,-3.407679,-5.063118,1.007042,-3.190158,-4.250717,...,1.059737,-0.037395,0.348707,-0.162929,0.410531,-0.123612,0.877424,0.667568,8.30,1


In [25]:
# seperate the features and target variables
labels = oversampled_data.columns[:30]
X = oversampled_data[labels]
y = oversampled_data['Target_flag']

# create train-test split with stratify
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7, stratify=y)

In [26]:
# check shape of created train-test split
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(367425, 30)
(40825, 30)
(367425,)
(40825,)


In [27]:
# training the model
model = build_model(X_train, metrics=metrics)

history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=20,
    validation_split=0.05,
    shuffle=True,
    verbose=1
)

Epoch 1/20
171/171 [==============================] - 2s 15ms/step - loss: 0.2982 - tp: 148096.0000 - fp: 15704.0000 - tn: 181550.0000 - fn: 26423.0000 - accuracy: 0.8867 - precision: 0.9041 - recall: 0.8486 - auc: 0.9503 - val_loss: 0.1450 - val_tp: 8863.0000 - val_fp: 579.0000 - val_tn: 8561.0000 - val_fn: 369.0000 - val_accuracy: 0.9484 - val_precision: 0.9387 - val_recall: 0.9600 - val_auc: 0.9903
Epoch 2/20
171/171 [==============================] - 1s 8ms/step - loss: 0.0867 - tp: 168511.0000 - fp: 4601.0000 - tn: 169972.0000 - fn: 5969.0000 - accuracy: 0.9697 - precision: 0.9734 - recall: 0.9658 - auc: 0.9950 - val_loss: 0.0504 - val_tp: 9005.0000 - val_fp: 171.0000 - val_tn: 8969.0000 - val_fn: 227.0000 - val_accuracy: 0.9783 - val_precision: 0.9814 - val_recall: 0.9754 - val_auc: 0.9984
Epoch 3/20
171/171 [==============================] - 1s 8ms/step - loss: 0.0218 - tp: 173355.0000 - fp: 1176.0000 - tn: 173397.0000 - fn: 1125.0000 - accuracy: 0.9934 - precision: 0.9933 - rec

In [28]:
# evaluate model performance on unseen data
model.evaluate(X_test, y_test, batch_size=batch_size)

20/20 [==============================] - 0s 5ms/step - loss: 1.8236e-05 - tp: 20413.0000 - fp: 0.0000e+00 - tn: 20412.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000


[1.8236469259136356e-05, 20413.0, 0.0, 20412.0, 0.0, 1.0, 1.0, 1.0, 1.0]

We can see considerable improvements now in our model performance. But oversampling increases the likelihood of overfitting since it replicates the minority class events.

# Undersample Majority Class

Undersampling aims to balance class distribution by randomly eliminating majority class examples. This is done until the majority and minority class instances are balanced out.
We’ll remove samples from the majority class and balance the data this way.

In [29]:
# seperate the features and target variables
labels = df.columns[1:30]
X = df[labels]
y = df['Target_flag']


# create train-test split with stratify  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7, stratify=y)
X = pd.concat([X_train, y_train], axis=1)

# seperate the majority and minority classes
negative_class = X[X.Target_flag == 0]
positive_class = X[X.Target_flag == 1]

In [30]:
# length of majority class
print("Length of Majority Class :", len(negative_class))

Length of Majority Class : 204125


In [31]:
# downsampling majority class
negative_downsampled = resample(negative_class,
                          replace=False,
                          n_samples=len(positive_class),
                          random_state=7)

In [32]:
# length of downsampled majority class
print("Length of Downsampled Majority Class :", len(negative_downsampled))

Length of Downsampled Majority Class : 353


In [33]:
# combine negative dowsampled class and positive class
undersampled_data = pd.concat([negative_downsampled, positive_class], axis=0)

In [34]:
# check new data
undersampled_data

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,Feature_26,Feature_27,Feature_28,Feature_29,Target_flag
193797,1.906400,-0.573265,-1.009407,0.792257,-0.198142,-0.067769,-0.192298,0.030639,1.420956,-0.131447,...,-0.365444,-0.867700,0.272260,0.590118,-0.178745,0.074378,-0.050384,-0.045986,58.68,0
143015,1.031841,-0.480074,1.284097,0.551383,-1.234127,-0.180908,-0.823551,0.095998,2.307650,-0.679066,...,-0.001148,0.236531,0.007198,0.420524,0.016228,1.075350,-0.064337,0.018270,73.80,0
44633,1.208691,-0.224328,0.318862,-0.317760,-0.630465,-0.565430,-0.305962,0.096606,0.182844,0.001888,...,0.077888,0.180560,0.029645,0.283920,0.117552,1.455968,-0.110750,-0.019538,4.99,0
135008,2.017787,-0.933999,-0.125465,-1.514940,-1.165311,-0.303344,-1.153623,0.077438,2.591155,-1.016055,...,0.241294,0.968823,0.185530,0.723452,-0.284142,-0.210828,0.067739,-0.020727,14.72,0
172548,-0.817895,-0.621163,2.106390,-1.394239,-0.621589,2.778371,0.006265,0.554697,-0.574781,0.058164,...,0.180020,0.918703,0.010010,-1.331956,-0.318719,-0.168814,-0.031983,-0.182348,176.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52630,-0.419820,-1.155978,-2.092516,2.786750,0.736297,-0.167292,1.600027,-0.117427,-0.796954,-0.133950,...,0.480640,0.533517,1.284645,0.516131,-0.602941,-0.305024,-0.021363,0.129096,451.27,1
123402,-1.125092,3.682876,-6.556168,4.016731,-0.425571,-2.031210,-2.650137,1.131249,-2.946890,-4.816401,...,1.185580,1.348156,-0.053686,0.284122,-1.174469,-0.087832,0.718790,0.676216,0.76,1
181339,-0.443794,1.271395,1.206178,0.790371,0.418935,-0.848376,0.917691,-0.235511,-0.285692,-0.867900,...,0.119279,0.513479,-0.264243,0.443311,0.029516,-0.335141,-0.188815,-0.123391,5.09,1
165973,-5.488032,3.329561,-5.996296,3.601720,-2.023926,-1.737393,-4.396859,0.228394,-1.675884,-3.991785,...,1.719631,0.343209,0.133584,0.833340,-0.839776,0.502010,-1.937473,1.521218,0.01,1


In [35]:
# seperate the features and target variables
labels = undersampled_data.columns[:30]
X = undersampled_data[labels]
y = undersampled_data['Target_flag']

# create train-test split with stratify
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7, stratify=y)

In [36]:
# check shape of created train-test split
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(635, 30)
(71, 30)
(635,)
(71,)


In [37]:
# training the model
model = build_model(X_train, metrics=metrics)

history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=400,
    validation_split=0.05,
    shuffle=True,
    verbose=1
)

Epoch 1/400
1/1 [==============================] - 1s 824ms/step - loss: 0.9075 - tp: 20548.0000 - fp: 159.0000 - tn: 20559.0000 - fn: 162.0000 - accuracy: 0.9923 - precision: 0.9923 - recall: 0.9922 - auc: 0.9998 - val_loss: 0.6319 - val_tp: 19.0000 - val_fp: 11.0000 - val_tn: 1.0000 - val_fn: 1.0000 - val_accuracy: 0.6250 - val_precision: 0.6333 - val_recall: 0.9500 - val_auc: 0.7000
Epoch 2/400
1/1 [==============================] - 0s 26ms/step - loss: 0.9030 - tp: 125.0000 - fp: 165.0000 - tn: 141.0000 - fn: 172.0000 - accuracy: 0.4411 - precision: 0.4310 - recall: 0.4209 - auc: 0.4516 - val_loss: 0.6289 - val_tp: 19.0000 - val_fp: 11.0000 - val_tn: 1.0000 - val_fn: 1.0000 - val_accuracy: 0.6250 - val_precision: 0.6333 - val_recall: 0.9500 - val_auc: 0.7000
Epoch 3/400
1/1 [==============================] - 0s 25ms/step - loss: 0.9352 - tp: 126.0000 - fp: 156.0000 - tn: 150.0000 - fn: 171.0000 - accuracy: 0.4577 - precision: 0.4468 - recall: 0.4242 - auc: 0.4369 - val_loss: 0.6211

1/1 [==============================] - 0s 27ms/step - loss: 0.5590 - tp: 195.0000 - fp: 49.0000 - tn: 257.0000 - fn: 102.0000 - accuracy: 0.7496 - precision: 0.7992 - recall: 0.6566 - auc: 0.8216 - val_loss: 0.5021 - val_tp: 20.0000 - val_fp: 6.0000 - val_tn: 6.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.8125 - val_precision: 0.7692 - val_recall: 1.0000 - val_auc: 0.9583
Epoch 23/400
1/1 [==============================] - 0s 26ms/step - loss: 0.4969 - tp: 202.0000 - fp: 41.0000 - tn: 265.0000 - fn: 95.0000 - accuracy: 0.7745 - precision: 0.8313 - recall: 0.6801 - auc: 0.8517 - val_loss: 0.4972 - val_tp: 20.0000 - val_fp: 6.0000 - val_tn: 6.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.8125 - val_precision: 0.7692 - val_recall: 1.0000 - val_auc: 0.9625
Epoch 24/400
1/1 [==============================] - 0s 41ms/step - loss: 0.4996 - tp: 216.0000 - fp: 57.0000 - tn: 249.0000 - fn: 81.0000 - accuracy: 0.7711 - precision: 0.7912 - recall: 0.7273 - auc: 0.8450 - val_loss: 0.4920 - val_tp: 20.

1/1 [==============================] - 0s 26ms/step - loss: 0.3776 - tp: 241.0000 - fp: 37.0000 - tn: 269.0000 - fn: 56.0000 - accuracy: 0.8458 - precision: 0.8669 - recall: 0.8114 - auc: 0.9118 - val_loss: 0.3978 - val_tp: 20.0000 - val_fp: 5.0000 - val_tn: 7.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.8438 - val_precision: 0.8000 - val_recall: 1.0000 - val_auc: 0.9958
Epoch 44/400
1/1 [==============================] - 0s 25ms/step - loss: 0.3673 - tp: 233.0000 - fp: 28.0000 - tn: 278.0000 - fn: 64.0000 - accuracy: 0.8474 - precision: 0.8927 - recall: 0.7845 - auc: 0.9195 - val_loss: 0.3946 - val_tp: 20.0000 - val_fp: 5.0000 - val_tn: 7.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.8438 - val_precision: 0.8000 - val_recall: 1.0000 - val_auc: 0.9917
Epoch 45/400
1/1 [==============================] - 0s 26ms/step - loss: 0.3754 - tp: 239.0000 - fp: 35.0000 - tn: 271.0000 - fn: 58.0000 - accuracy: 0.8458 - precision: 0.8723 - recall: 0.8047 - auc: 0.9145 - val_loss: 0.3912 - val_tp: 20.0

1/1 [==============================] - 0s 31ms/step - loss: 0.3396 - tp: 245.0000 - fp: 29.0000 - tn: 277.0000 - fn: 52.0000 - accuracy: 0.8657 - precision: 0.8942 - recall: 0.8249 - auc: 0.9261 - val_loss: 0.3288 - val_tp: 20.0000 - val_fp: 3.0000 - val_tn: 9.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9062 - val_precision: 0.8696 - val_recall: 1.0000 - val_auc: 0.9958
Epoch 65/400
1/1 [==============================] - 0s 29ms/step - loss: 0.3115 - tp: 246.0000 - fp: 25.0000 - tn: 281.0000 - fn: 51.0000 - accuracy: 0.8740 - precision: 0.9077 - recall: 0.8283 - auc: 0.9401 - val_loss: 0.3260 - val_tp: 20.0000 - val_fp: 3.0000 - val_tn: 9.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9062 - val_precision: 0.8696 - val_recall: 1.0000 - val_auc: 0.9937
Epoch 66/400
1/1 [==============================] - 0s 30ms/step - loss: 0.3143 - tp: 248.0000 - fp: 21.0000 - tn: 285.0000 - fn: 49.0000 - accuracy: 0.8839 - precision: 0.9219 - recall: 0.8350 - auc: 0.9354 - val_loss: 0.3236 - val_tp: 20.0

1/1 [==============================] - 0s 27ms/step - loss: 0.2368 - tp: 263.0000 - fp: 16.0000 - tn: 290.0000 - fn: 34.0000 - accuracy: 0.9171 - precision: 0.9427 - recall: 0.8855 - auc: 0.9660 - val_loss: 0.2739 - val_tp: 20.0000 - val_fp: 2.0000 - val_tn: 10.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9375 - val_precision: 0.9091 - val_recall: 1.0000 - val_auc: 0.9917
Epoch 86/400
1/1 [==============================] - 0s 27ms/step - loss: 0.2588 - tp: 259.0000 - fp: 19.0000 - tn: 287.0000 - fn: 38.0000 - accuracy: 0.9055 - precision: 0.9317 - recall: 0.8721 - auc: 0.9550 - val_loss: 0.2710 - val_tp: 20.0000 - val_fp: 2.0000 - val_tn: 10.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9375 - val_precision: 0.9091 - val_recall: 1.0000 - val_auc: 0.9917
Epoch 87/400
1/1 [==============================] - 0s 27ms/step - loss: 0.2593 - tp: 261.0000 - fp: 24.0000 - tn: 282.0000 - fn: 36.0000 - accuracy: 0.9005 - precision: 0.9158 - recall: 0.8788 - auc: 0.9594 - val_loss: 0.2686 - val_tp: 20

...

1/1 [==============================] - 0s 27ms/step - loss: 0.0269 - tp: 295.0000 - fp: 1.0000 - tn: 305.0000 - fn: 2.0000 - accuracy: 0.9950 - precision: 0.9966 - recall: 0.9933 - auc: 0.9998 - val_loss: 0.1336 - val_tp: 17.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 3.0000 - val_accuracy: 0.9062 - val_precision: 1.0000 - val_recall: 0.8500 - val_auc: 0.9917
Epoch 317/400
1/1 [==============================] - 0s 25ms/step - loss: 0.0285 - tp: 296.0000 - fp: 3.0000 - tn: 303.0000 - fn: 1.0000 - accuracy: 0.9934 - precision: 0.9900 - recall: 0.9966 - auc: 0.9996 - val_loss: 0.1254 - val_tp: 17.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 3.0000 - val_accuracy: 0.9062 - val_precision: 1.0000 - val_recall: 0.8500 - val_auc: 0.9958
Epoch 318/400
1/1 [==============================] - 0s 27ms/step - loss: 0.0291 - tp: 296.0000 - fp: 4.0000 - tn: 302.0000 - fn: 1.0000 - accuracy: 0.9917 - precision: 0.9867 - recall: 0.9966 - auc: 0.9998 - val_loss: 0.1185 - val_tp: 17.000

1/1 [==============================] - 0s 25ms/step - loss: 0.0423 - tp: 292.0000 - fp: 2.0000 - tn: 304.0000 - fn: 5.0000 - accuracy: 0.9884 - precision: 0.9932 - recall: 0.9832 - auc: 0.9989 - val_loss: 0.2043 - val_tp: 17.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 3.0000 - val_accuracy: 0.9062 - val_precision: 1.0000 - val_recall: 0.8500 - val_auc: 0.9917
Epoch 338/400
1/1 [==============================] - 0s 25ms/step - loss: 0.0382 - tp: 292.0000 - fp: 2.0000 - tn: 304.0000 - fn: 5.0000 - accuracy: 0.9884 - precision: 0.9932 - recall: 0.9832 - auc: 0.9991 - val_loss: 0.2036 - val_tp: 17.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 3.0000 - val_accuracy: 0.9062 - val_precision: 1.0000 - val_recall: 0.8500 - val_auc: 0.9917
Epoch 339/400
1/1 [==============================] - 0s 25ms/step - loss: 0.0292 - tp: 293.0000 - fp: 3.0000 - tn: 303.0000 - fn: 4.0000 - accuracy: 0.9884 - precision: 0.9899 - recall: 0.9865 - auc: 0.9997 - val_loss: 0.2035 - val_tp: 17.000

1/1 [==============================] - 0s 27ms/step - loss: 0.0419 - tp: 290.0000 - fp: 1.0000 - tn: 305.0000 - fn: 7.0000 - accuracy: 0.9867 - precision: 0.9966 - recall: 0.9764 - auc: 0.9974 - val_loss: 0.2472 - val_tp: 17.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 3.0000 - val_accuracy: 0.9062 - val_precision: 1.0000 - val_recall: 0.8500 - val_auc: 0.9917
Epoch 359/400
1/1 [==============================] - 0s 26ms/step - loss: 0.0275 - tp: 292.0000 - fp: 1.0000 - tn: 305.0000 - fn: 5.0000 - accuracy: 0.9900 - precision: 0.9966 - recall: 0.9832 - auc: 0.9997 - val_loss: 0.2034 - val_tp: 17.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 3.0000 - val_accuracy: 0.9062 - val_precision: 1.0000 - val_recall: 0.8500 - val_auc: 0.9917
Epoch 360/400
1/1 [==============================] - 0s 28ms/step - loss: 0.0292 - tp: 295.0000 - fp: 0.0000e+00 - tn: 306.0000 - fn: 2.0000 - accuracy: 0.9967 - precision: 1.0000 - recall: 0.9933 - auc: 0.9994 - val_loss: 0.1500 - val_tp: 17

1/1 [==============================] - 0s 27ms/step - loss: 0.0389 - tp: 295.0000 - fp: 5.0000 - tn: 301.0000 - fn: 2.0000 - accuracy: 0.9884 - precision: 0.9833 - recall: 0.9933 - auc: 0.9988 - val_loss: 0.0613 - val_tp: 19.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 1.0000 - val_accuracy: 0.9688 - val_precision: 1.0000 - val_recall: 0.9500 - val_auc: 1.0000
Epoch 380/400
1/1 [==============================] - 0s 25ms/step - loss: 0.0350 - tp: 294.0000 - fp: 3.0000 - tn: 303.0000 - fn: 3.0000 - accuracy: 0.9900 - precision: 0.9899 - recall: 0.9899 - auc: 0.9991 - val_loss: 0.0642 - val_tp: 19.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 1.0000 - val_accuracy: 0.9688 - val_precision: 1.0000 - val_recall: 0.9500 - val_auc: 1.0000
Epoch 381/400
1/1 [==============================] - 0s 26ms/step - loss: 0.0402 - tp: 294.0000 - fp: 2.0000 - tn: 304.0000 - fn: 3.0000 - accuracy: 0.9917 - precision: 0.9932 - recall: 0.9899 - auc: 0.9971 - val_loss: 0.1083 - val_tp: 17.000

1/1 [==============================] - 0s 37ms/step - loss: 0.0215 - tp: 294.0000 - fp: 2.0000 - tn: 304.0000 - fn: 3.0000 - accuracy: 0.9917 - precision: 0.9932 - recall: 0.9899 - auc: 0.9998 - val_loss: 0.2066 - val_tp: 17.0000 - val_fp: 0.0000e+00 - val_tn: 12.0000 - val_fn: 3.0000 - val_accuracy: 0.9062 - val_precision: 1.0000 - val_recall: 0.8500 - val_auc: 1.0000


In [38]:
# evaluate model performance on unseen data
model.evaluate(X_test, y_test, batch_size=batch_size)

1/1 [==============================] - 0s 3ms/step - loss: 0.1619 - tp: 32.0000 - fp: 1.0000 - tn: 34.0000 - fn: 4.0000 - accuracy: 0.9296 - precision: 0.9697 - recall: 0.8889 - auc: 0.9825


[0.161886066198349,
 32.0,
 1.0,
 34.0,
 4.0,
 0.9295774698257446,
 0.9696969985961914,
 0.8888888955116272,
 0.982539713382721]

We finally achieve good results across all metrics. We save the weights of this model into .h5 file 

In [39]:
# save the model weights to .h5 file
model.save_weights('trained_weights_400eps_tauc0.98.h5')

In [40]:
# save model
model.save('trained_model_object.h5')